# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [1]:
import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, concat, desc, explode, lit, min, max, split, udf, lag
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.functions import countDistinct
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, LinearSVC
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import CountVectorizer, IDF, Normalizer, PCA, RegexTokenizer, StandardScaler, StopWordsRemover, StringIndexer, VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.sql.window import Window

import numpy as np

#%matplotlib inline
#import matplotlib.pyplot as plt

import re


VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1620765055747_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# Create spark session
spark = SparkSession \
    .builder \
    .appName("Sparkify-flavio3") \
    .getOrCreate()



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Read in full sparkify dataset
event_data = "s3n://udacity-dsnd/sparkify/sparkify_event_data.json"
#event_data = "s3n://udacity-dsnd/sparkify/mini_sparkify_event_data.json"
df = spark.read.json(event_data)
df.persist()
df.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(artist='Popol Vuh', auth='Logged In', firstName='Shlok', gender='M', itemInSession=278, lastName='Johnson', length=524.32934, level='paid', location='Dallas-Fort Worth-Arlington, TX', method='PUT', page='NextSong', registration=1533734541000, sessionId=22683, song='Ich mache einen Spiegel - Dream Part 4', status=200, ts=1538352001000, userAgent='"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"', userId='1749042')

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [4]:
#file = 'mini_sparkify_event_data.json'
#df = spark.read.json(file)

df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)

In [5]:
print('number of rows:' , df.count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

number of rows: 26259199

In [6]:
# clean rows with empty userId
df = df.filter("userId <> ''")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
df.select("userId").distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

22278

In [9]:
#userId_count_pd = df.groupby("userId").count().sort("count", ascending=[False]).toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
#userId_count_pd['count'].plot(kind='hist', bins=50)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
#userId_count_pd.describe()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
#userId_count_pd=None

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
df.select("level").distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+
|level|
+-----+
| free|
| paid|
+-----+

In [13]:
df.select("page").distinct().show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------+
|page                     |
+-------------------------+
|Cancel                   |
|Submit Downgrade         |
|Thumbs Down              |
|Home                     |
|Downgrade                |
|Roll Advert              |
|Logout                   |
|Save Settings            |
|Cancellation Confirmation|
|About                    |
|Submit Registration      |
|Settings                 |
|Login                    |
|Register                 |
|Add to Playlist          |
|Add Friend               |
|NextSong                 |
|Thumbs Up                |
|Help                     |
|Upgrade                  |
+-------------------------+
only showing top 20 rows

In [14]:
print('number of disctinct locations: ',
      df.select('location').distinct().count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

number of disctinct locations:  887

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [15]:
hasCancelled = udf(lambda x: 1 if x=='Cancellation Confirmation' else 0, IntegerType())
df = df.withColumn('Churn', hasCancelled(df['page']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
user_window = Window.partitionBy('userId')

df = df.withColumn('Churned', F.max('Churn').over(user_window))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
print("total users churned",
      df.select(['userId']).filter("Churn=1").distinct().count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

total users churned 5003

In [12]:
print('churn rate: ',52/226)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

churn rate:  0.23008849557522124

In [18]:
df.select(["artist"]).distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

38338

In [19]:
is_song = udf(lambda x: 1 if x=='NextSong' else 0, IntegerType())
df = df.withColumn('NextSong', is_song(df['page']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
has_paid= udf(lambda x: 1 if x=='paid' else 0, IntegerType()) 
df = df.withColumn('hasPaid', has_paid('level'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
has_downgrade =  udf(lambda x: 1 if x=='Downgrade' else 0, IntegerType()) 
df = df.withColumn('hasDowngrade', has_downgrade('page'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
"""
test_artist = df.filter("artist is not null").groupBy(["artist"]).count().sort('count', ascending=[False])\
    .withColumn('percent', F.col('count')/F.sum('count').over(Window.partitionBy())*100)\
    .withColumn('cum_percent', F.sum('percent').over(Window.partitionBy().orderBy().rowsBetween(Window.unboundedPreceding, 0)))\

#test_artist.show(1700)

artist_no_churned = np.array(df.filter('artist is not null and Churned=0').select('artist').collect())
artist_no_churned = set(artist_no_churned.flatten())

artist_churned = np.array(df.filter('artist is not null and Churned=1').select('artist').collect())
artist_churned = set(artist_churned.flatten())

artist_only_churn = artist_churned - artist_no_churned
artist_only_no_churn = artist_no_churned - artist_churned

print('distinct artist: ',
      '17655')
      #df.filter('artist is not null').select('artist').distinct().count())

print('distinct artist in churned users: ',
      len(artist_churned))

print('distinct artist not churned users: ',
      len(artist_no_churned))

print('artist only in churned: ', 
      len(artist_churned - artist_no_churned))

print('artist only in not churned: ',
     len(artist_no_churned - artist_churned))

df = df.withColumn('artist_only_churn', col('artist').isin(artist_only_churn).cast('integer'))
df = df.withColumn('artist_only_no_churn', col('artist').isin(artist_only_no_churn).cast('integer')) 

"""

distinct artist:  17655
distinct artist in churned users:  8402
distinct artist not churned users:  16652
artist only in churned:  1003
artist only in not churned:  9253


#### general info aggregated by userId

In [22]:
# general info aggregated by userId
user = df.groupBy("userId").agg(
    F.max('location').alias('max_location'),
    F.max('gender').alias('max_gender'),
    F.max('hasPaid').alias('max_hasPaid'),
    F.max('hasDowngrade').alias('max_hasDowngrade'),
    F.max('Churn').alias('max_churn'),
    F.countDistinct('artist').alias('dist_artist')
#    F.collect_list("artist").alias("artist_list")
)

#user.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### time between user sessions

In [23]:
# time between sessionsa

# rank in session
session_by_ts_win = Window.partitionBy(['userId','sessionId']).orderBy('ts')
df = df.withColumn('rank', F.rank().over(session_by_ts_win))

# time_diff
user_by_ts = Window.partitionBy(['userId']).orderBy('ts')
df = df.withColumn('ts_diff', col('ts') - F.lag('ts',1).over(user_by_ts))

# time diff between first page in current session - last page in previous session
time_btw_session = df.select(['userId','ts_diff'])\
                         .filter("rank=1 and ts_diff is not null")\
                         .groupBy('userId').agg(avg('ts_diff').alias('time_btw_sessions'))
#time_btw_session.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### number of songs played, and number os sessions


In [24]:
# number of songs played, and number os sessions

song_session = df.groupBy(['userId','sessionId'])\
                .agg(
                    F.sum('length'), 
                    F.sum('NextSong')\
                )\
                .groupBy('userId')\
                .agg(
                    F.avg('sum(length)').alias("avg_session_length"),
                    F.avg('sum(NextSong)').alias("avg_session_songs"),
                    F.sum('sum(length)').alias("total_session_length"),   
                    F.sum('sum(NextSong)').alias("total_songs_played"),
                    F.count('sum(length)').alias("total_sessions")
                )

#song_session.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### join aggregated columns

In [25]:
joined_df = None
joined_df = user.join(song_session, ["userId"], "inner")\
                .join(time_btw_session, ["userId"], "inner")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### persist processed dataframe and save to file

In [26]:
user2 = joined_df.persist()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
#user2.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error occurred while calling o459.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 45.0 failed 4 times, most recent failure: Lost task 2.3 in stage 45.0 (TID 1504, ip-172-31-26-242.ec2.internal, executor 1): java.lang.RuntimeException: Failed to run command: /usr/bin/virtualenv -p python3 --system-site-packages virtualenv_application_1620762085855_0003_0
	at org.apache.spark.api.python.VirtualEnvFactory.execCommand(VirtualEnvFactory.scala:120)
	at org.apache.spark.api.python.VirtualEnvFactory.setupVirtualEnv(VirtualEnvFactory.scala:78)
	at org.apache.spark.api.python.PythonWorkerFactory.<init>(PythonWorkerFactory.scala:85)
	at org.apache.spark.SparkEnv.$anonfun$createPythonWorker$1(SparkEnv.scala:118)
	at scala.collection.mutable.HashMap.getOrElseUpdate(HashMap.scala:86)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:118)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:109)
	at org.apache.sp

In [32]:
#user2.write.mode("overwrite").parquet("user-full.parquet")

In [27]:
user2.write.mode("overwrite").parquet("s3n://analitics-flavio/user-full.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
#user_pd = user2.toPandas()

In [34]:
#user_pd.groupby('max_churn').mean()

,max_hasPaid,max_hasDowngrade,dist_artist,avg_session_length,avg_session_songs,total_session_length,total_songs_played,total_sessions,time_btw_sessions
max_churn,,,,,,,,,
0,0.768293,0.719512,786.993902,18132.491619,70.911949,290287.935539,1164.829268,15.506098,5.207053e+08
1,0.760870,0.739130,581.456522,16624.929701,64.868798,195489.245545,786.173913,11.152174,2.500782e+08


# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

In [28]:
df2 = spark.read.parquet("s3n://analitics-flavio/user-full.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
df2 = df2.withColumnRenamed('max_churn','label')


df2.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- userId: string (nullable = true)
 |-- max_location: string (nullable = true)
 |-- max_gender: string (nullable = true)
 |-- max_hasPaid: integer (nullable = true)
 |-- max_hasDowngrade: integer (nullable = true)
 |-- label: integer (nullable = true)
 |-- dist_artist: long (nullable = true)
 |-- avg_session_length: double (nullable = true)
 |-- avg_session_songs: double (nullable = true)
 |-- total_session_length: double (nullable = true)
 |-- total_songs_played: long (nullable = true)
 |-- total_sessions: long (nullable = true)
 |-- time_btw_sessions: double (nullable = true)

In [42]:
validation, test = df2.randomSplit([0.9, 0.1], seed=42)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
validation.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

19078

In [54]:
def fit_model(model, paramGrid):
#cv = CountVectorizer(inputCol="artist_list", outputCol="TF")
#location_index = StringIndexer(inputCol='max_location', outputCol="location_index", handleInvalid='skip')

    gender_index = StringIndexer(inputCol='max_gender', outputCol="gender_index", handleInvalid='skip')

    #scaler
    numerical = ["max_hasDowngrade","max_hasPaid","dist_artist", "avg_session_length",
                 "avg_session_songs","total_session_length","total_songs_played","total_sessions","time_btw_sessions"]

    numfeatures = VectorAssembler(inputCols=numerical, outputCol='NumFeatures')
    scaler = StandardScaler(inputCol='NumFeatures', outputCol="ScaledNumFeatures")

    assembler = VectorAssembler(inputCols=[#"TF",
                                           #"location_index",
                                           "gender_index",
                                           "ScaledNumFeatures"
                                          ], 
                                outputCol='features')

    

    pipeline = Pipeline(stages=[gender_index,
                                numfeatures, 
                                scaler, 
                                assembler,
                                model])

    crossval = CrossValidator(estimator=pipeline,
                              estimatorParamMaps=paramGrid,
                              evaluator=MulticlassClassificationEvaluator(metricName='f1'),
                              numFolds=3)

    fitted_model = crossval.fit(validation)
    
    return fitted_model

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
def evaluate_results(results):
    score = MulticlassClassificationEvaluator(metricName='f1').evaluate(results)
    return score

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
############################
# Logistic Regression
lr=  LogisticRegression(maxIter=10, regParam=0.0, elasticNetParam=0)

lr_grid = ParamGridBuilder() \
    .addGrid(lr.regParam,[0.0, 0.1]) \
    .build()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [57]:
model1 = fit_model(lr, lr_grid)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [58]:
model1.avgMetrics

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[0.809313177935544, 0.6881400466262212]

In [59]:
test_results = model1.transform(test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [60]:
evaluate_results(model1.transform(test))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.8051695531178079

In [61]:
# RandomForestClassifier
rf = RandomForestClassifier()
rf_grid = ParamGridBuilder() \
    .addGrid(rf.maxDepth,[4,10,20])\
    .addGrid(rf.numTrees,[15,30])\
    .build()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [62]:
model2 = fit_model(rf, rf_grid)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [63]:
model2.avgMetrics

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[0.8254281412429008, 0.8232275130772798, 0.8810814620140055, 0.8795643722525632, 0.8757702182385305, 0.8760262149640596]

In [64]:
evaluate_results(model2.transform(test))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.8790249049345811

In [65]:
# LinearSVC
lsvc = LinearSVC()
lsvc_grid = ParamGridBuilder() \
    .addGrid(lsvc.maxIter,[10,20])\
    .addGrid(lsvc.regParam,[0.0,0.1])\
    .build()

model3 = fit_model(lsvc, lsvc_grid)
evaluate_results(model3.transform(test))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-65:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 1399



0.6797480399168581

In [66]:
model3.avgMetrics

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[0.6880135436724973, 0.6880135436724973, 0.6880135436724973, 0.6880135436724973]

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.